# 数据集和数据加载器

- 译文：https://pytorch.apachecn.org/2.0/tutorials/beginner/basics/data_tutorial
- 原文：https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

## 概要

- 问题：处理数据样本的代码容易变得混乱，难以维护。
- 目标：将数据逻辑与训练逻辑分离，提高可读性与模块化。
- PyTorch 提供两个基类：`torch.utils.data.Dataset`（表示数据集，负责单样本读取与转换）与 `torch.utils.data.DataLoader`（在 Dataset 之上提供可迭代的批次、shuffle、并行加载等）。
- 同时也有预置数据集（如 FashionMNIST），可用于原型与基准测试。

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
# 从 torchvision 加载 FashionMNIST 数据集
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

print('Training samples:', len(training_data))
print('Test samples:', len(test_data))

## 迭代与可视化数据集

- 你可以像访问列表一样通过索引访问 `Dataset`（例如 `training_data[idx]`）。下面演示如何随机抽取样本并用 `matplotlib` 可视化。

In [ ]:
labels_map = {
    0: 'T-Shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot',
}

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis('off')
    plt.imshow(img.squeeze(), cmap='gray')
plt.show()

## 为你的文件创建自定义 Dataset

- 自定义 Dataset 必须实现三个方法：`__init__`, `__len__`, `__getitem__`。
- 示例场景：图像文件存储在某个目录，标签保存在 CSV 中。下面给出一个可复用的模板类。

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = int(self.img_labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

# 使用示例（不运行）：
# dataset = CustomImageDataset('annotations.csv', 'img_dir')
# img, label = dataset[0]

### 细节说明

- `__init__`：在实例化时运行一次，用于加载标签文件、设置图像目录与转换函数。示例 CSV 行格式：`tshirt1.jpg,0`。
- `__len__`：返回数据集大小，通常为标签表的行数。
- `__getitem__`：根据索引读取图像文件并返回 `(image_tensor, label)`，在返回前可应用 `transform` 与 `target_transform`。

## 使用 DataLoader 为训练准备数据

- 在训练中常使用小批量（batch）、每 epoch 洗牌（shuffle）和多进程加载（`num_workers`）来加速训练与提升泛化能力。
- `DataLoader` 提供简洁的 API 来实现这些功能。

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader  = DataLoader(test_data, batch_size=64, shuffle=True)

print('Created DataLoaders: train ->', len(train_dataloader), 'batches; test ->', len(test_dataloader), 'batches')

In [ ]:
# 遍历 DataLoader：每个迭代返回一个 batch 的 features 和 labels
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

# 可视化第一个样本
img = train_features[0].squeeze()
label = train_labels[0].item()
plt.imshow(img, cmap='gray')
plt.title(f'Label: {label}')
plt.axis('off')
plt.show()